 \\[
 \begin{aligned}
 \frac{dS}{dt} &= -\frac{\beta SI}{N} \\
 \frac{dI}{dt} &= \frac{\beta SI}{N} - \gamma I\\
 \frac{dR}{dt} &= \gamma I 
 \end{aligned}
 \\]

In [1]:
import numpy as np
from scipy.optimize import minimize
import plotly.graph_objects as go 
import plotly.express as px 
import plotly.io as pio
from plotly.subplots import make_subplots

In [2]:
from abc import ABCMeta, abstractmethod

class Law(metaclass=ABCMeta):
    @staticmethod
    @abstractmethod
    def sample(n, d):
        pass

    @staticmethod
    @abstractmethod
    def loglikely(n, d, k):
        pass
    
    @staticmethod
    def likelihood(n, d, k):
        return np.exp(loglikely(cls, n, d, k))
    
class Bin(Law):
    def sample(n, d):
        return np.random.binomial(n, d)
    
    def loglikely(n, d, k):
        return k*np.log(d) + (n-k)*np.log(1-d)
        
class Poi(Law):
    def sample(n, d):
        return np.random.poisson(n * d)
    
    def loglikely(n, d, k):
        return k*np.log(n*d) - n*d

In [3]:
class Region:
    def __init__(self, S, I, R):
        self.S = S
        self.I = I
        self.R = R
        self.N = S + I + R
        
    def __repr__(self):
        return "S={}, I={}, R={}".format(self.S, self.I, self.R)
    
class SIR:
    def __init__(self, beta, gamma, dt=1):
        self.beta = beta * dt
        self.gamma = gamma * dt
        
    def __repr__(self):
        return "β={}, γ={}".format(self.beta, self.gamma)
    
class Epidemic:
    def __init__(self, region, dynamic, T):
        S = np.zeros(T+1)
        I = np.zeros(T+1)
        R = np.zeros(T+1)
        S[0] = region.S
        I[0] = region.I
        R[0] = region.R

        for t in range(T):
            a, b = dynamic.beta*S[t]*I[t]/region.N, dynamic.gamma*I[t]
            S[t+1] = S[t] - a
            I[t+1] = I[t] + a - b
            R[t+1] = R[t] + b
            
        self.S = S
        self.I = I
        self.R = R
        self.N = region.N
        self.T = T
        
    def plot(self):
        fig = go.Figure()
        fig.update_layout(margin=dict(b=0, l=0, r=0, t=25))
        T = self.T
        fig.add_scatter(x=np.arange(T+1), y=self.S.astype(int), name="Susceptible", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.I.astype(int), name="Infectious", hovertemplate="%{y}")
        fig.add_scatter(x=np.arange(T+1), y=self.R.astype(int), name="Removed", hovertemplate="%{y}")
        return fig
    
class Sample:
    def __init__(self, epidemic, ts, ns, law):
        positive = np.zeros_like(ts)
        for i, (t, n) in enumerate(zip(ts, ns)):
            positive[i] = law.sample(n, epidemic.I[t]/epidemic.N)
            
        self.t = ts
        self.n = ns
        self.positive = positive
        self._law = law
        
    def __repr__(self):
        return " t: {} \n n: {} \n positive: {}".format(self.t, self.n, self.positive)
    
    def plot(self, epidemic):
        fig = epidemic.plot()
        fig.add_scatter(
            x=self.t, y=self.positive/self.n*epidemic.N, 
            mode="markers", name="Guessed", hovertemplate="%{y}"
        )
        return fig

In [54]:
def loglikely(epidemic, sample, law):
    ns = sample.n
    ds = epidemic.I[sample.t] / epidemic.N
    ks = sample.positive
    return sum(law.loglikely(n, d, k) for n, d, k in zip(ns, ds, ks))

def likelihood(epidemic, sample, law):
    return np.exp(loglikely(epidemic, sample, law))

identity = lambda x: x

one = lambda x: 1

class InferSIR():
    def __init__(self, law="binomial", algo="map"):
        self.law = law
        self.algo = algo
        
    def __str__(self):
        return "β={}, γ={}".format(self.beta, self.gamma)
    
    def plot(self, region, sample, law=None):
        if law is None:
            law = self.law
                    
        x, y = np.logspace(-2, 0, 30), np.logspace(-2, 0, 30)
        z = np.zeros((len(y), len(x)))
        for i in range(len(y)):
            for j in range(len(x)):
                dynamic = SIR(x[j], y[i])
                epidemic = Epidemic(region, dynamic, sample.t[-1])
                z[len(x)-1-i, j] = loglikely(epidemic, sample, law)
                
        fig = go.Figure(data=go.Contour(z=np.log(-z), x=x, y=y))
        fig.update_layout(
            margin=dict(b=0, l=0, r=0, t=25),
            xaxis=dict(scaleanchor="y", scaleratio=1, constrain="domain", range=(-2, 0))
        )
        fig.update_xaxes(type="log")
        fig.update_yaxes(type="log")
        return fig
        
    def fit_beta_gamma_map(self, region, sample, law=None, **kvarg):
        if law is None:
            law = self.law
            
        def func(x):
            dynamic = SIR(x[0], x[1])
            epidemic = Epidemic(region, dynamic, sample.t[-1])
            return -loglikely(epidemic, sample, law)
        
        res = minimize(func, (0.5,0.5), method='nelder-mead', options={'xatol': 1e-8, 'disp': True})
        self.beta, self.gamma = res.x
        fig = self.plot(region, sample, law)
        fig.add_scatter(x=[self.beta], y=[self.gamma])
        fig.show()
        
    def fit_beta_gamma_mcmc(self, region, sample, law=None, N=1000, 
                            mirror=(identity, identity), trans=(identity, one, identity),
                            var=[[0.01, 0],[0, 0.01]], **kvarg):
        if law is None:
            law = self.law
            
        def func(x):
            dynamic = SIR(trans[0](x[0]), trans[0](x[1]))
            epidemic = Epidemic(region, dynamic, sample.t[-1])
            return likelihood(epidemic, sample, law)*trans[1](x[0])*trans[1](x[1])
        
        walker = np.zeros((N+1, 2))
        x = trans[2](np.array([0.5, 0.5]))
        px = func(x)
        walker[0, :] = x
        for n in range(N):
            y = mirror[1](mirror[0](x) + np.random.multivariate_normal((0, 0), var))
            if y[0]<trans[2](0.001) or y[0]>trans[2](1) or y[1]<trans[2](0.001) or y[1]>trans[2](1):
                y = x
            py = func(y)
            if np.random.rand() < py/px:
                x, px = y, py
            walker[n+1, :] = x
            
        res = trans[0](np.mean(walker[100:, :], axis=0))
        self.beta = res[0]
        self.gamma = res[1]
        self.walker = trans[0](walker)
        
        fig = self.plot(region, sample, law)
        fig.add_scatter(x=self.walker[:, 0], y=self.walker[:, 1], mode="markers+lines")
        fig.show()        
    


In [55]:
T = 100
city = Region(9990, 10, 0)
epidemic = Epidemic(city, SIR(3, 1, 0.1), T)
np.random.seed(0)
sample = Sample(epidemic, np.arange(T//10, T, T//10), 10*np.ones(9, dtype=int), Poi)
sample.positive = np.array([0, 1, 2, 6, 1, 0, 0, 1, 0])
sample.plot(epidemic)

In [56]:
f = InferSIR()
f.fit_beta_gamma_map(city, sample, Bin)
print(f)

Optimization terminated successfully.
         Current function value: 25.845462
         Iterations: 72
         Function evaluations: 136


β=0.32553692171220483, γ=0.0884761552434795


In [57]:
np.random.seed(0)
g = InferSIR()
g.fit_beta_gamma_mcmc(city, sample, Bin, var=[[0.01, 0],[0, 0.01]])
print(g)

β=0.3419427000303564, γ=0.09550215574209563


In [58]:
np.random.seed(0)
g = InferSIR()
g.fit_beta_gamma_mcmc(city, sample, Bin, var=[[0.01, 0],[0, 0.001]])
print(g)

β=0.34112085510957857, γ=0.09561952075654649


In [59]:
np.random.seed(0)
g = InferSIR()
g.fit_beta_gamma_mcmc(city, sample, Bin, N=500, mirror=(np.log, np.exp), var=[[0.05, 0],[0, 0.05]])
print(g)

β=0.33433700391930676, γ=0.08681641782360743


In [60]:
np.random.seed(0)
g = InferSIR()
g.fit_beta_gamma_mcmc(city, sample, Bin, N=500, trans=(np.exp, np.exp, np.log), var=[[0.05, 0],[0, 0.05]])
print(g)

β=0.33465465208518475, γ=0.09060627938943011
